<a href="https://colab.research.google.com/github/EngLeandroA/MRMR_Dataset_EEG/blob/main/MRMR_Dataset_EEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
 pip install pymrmr

     |████████████████████████████████| 71kB 4.3MB/s 
  Created wheel for pymrmr: filename=pymrmr-0.1.11-cp37-cp37m-linux_x86_64.whl size=276546 sha256=456efcd1a70353ef6f2a85df6959685640594a3d63af216fda0fc501f87eda5c
  Stored in directory: /root/.cache/pip/wheels/9b/e2/c2/4b232fc0b8128263b518808866f5797070fe6750203ad1027a
Successfully built pymrmr


In [ ]:
import pymrmr

In [ ]:
# Biblioteca utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import operator
import random

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MaxAbsScaler ## Normalizar pelo maior

In [ ]:
colum = list()
for x in range(1, 4002):
  colum.append(f'T{x}')

In [ ]:
# Carregar o dataset do Github

url = "https://raw.githubusercontent.com/EngLeandroA/MRMR_Dataset_EEG/main/Dataset/V1/h2_v1_b_filtragem_p4_ref_sobi_CorrICA.csv?token=AO4ONDB7NDK7AQACX4NWALDAJDWYM"

dataset = pd.read_csv(url, sep=',', names=colum)

HTTPError: ignored

In [ ]:
dataset.shape

NameError: ignored

In [ ]:
dataset.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T22,T23,T24,T25,T26,T27,T28,T29,T30,T31,T32,T33,T34,T35,T36,T37,T38,T39,T40,...,T3962,T3963,T3964,T3965,T3966,T3967,T3968,T3969,T3970,T3971,T3972,T3973,T3974,T3975,T3976,T3977,T3978,T3979,T3980,T3981,T3982,T3983,T3984,T3985,T3986,T3987,T3988,T3989,T3990,T3991,T3992,T3993,T3994,T3995,T3996,T3997,T3998,T3999,T4000,T4001
0,1.308879,-4.956284,-7.472466,-4.395810,-3.742904,-4.769567,-5.463751,-3.053781,-2.692839,-3.461925,-3.936091,-4.458679,-5.322952,-6.552243,-5.435419,-3.946322,-1.973162,-1.290009,-1.012598,-1.061232,-1.712141,-2.022798,-3.792745,-4.323296,-5.371777,-5.526093,-6.359079,-6.568908,-7.558231,-8.499255,-9.453857,-9.555435,-8.481206,-8.072515,-8.346325,-7.698540,-6.443885,-5.756477,-6.697583,-7.519028,...,-9.474319,-5.469409,0.059101,3.732007,4.544501,4.422153,4.792804,4.381903,1.690433,-2.387122,-5.924873,-7.675067,-8.517274,-8.322786,-7.192645,-5.518717,-4.643095,-5.224376,-5.211134,-3.742492,-2.081722,-2.256799,-4.380442,-5.942301,-5.187319,-3.164403,-1.670954,-0.333837,1.423820,2.972975,3.874237,4.014191,4.056525,2.724800,0.024139,-1.618166,-0.539310,0.620100,-1.897345,-7.279020
1,4.197934,4.799670,5.811594,5.461148,7.432378,8.852448,10.386000,9.239245,8.696990,6.411046,3.707746,2.193855,1.045912,0.900191,0.653786,2.452090,4.141548,5.002052,5.734267,6.360171,6.740502,4.883553,3.798187,2.036639,2.212442,2.615790,4.083997,4.044909,3.714765,2.264302,0.837401,-0.486672,-1.783147,-1.739160,-0.327846,0.627577,1.424262,2.976894,4.670206,4.208098,...,-8.733079,-7.228108,-6.586715,-7.562081,-11.760860,-18.284990,-22.296490,-19.126810,-10.058450,-2.223038,-0.977000,-5.374013,-8.983285,-9.300155,-7.647831,-7.892128,-9.548452,-8.839690,-4.876411,-1.468578,-2.755864,-6.978095,-7.978503,-4.256879,-1.530791,-4.827703,-11.433960,-14.590060,-12.482270,-9.477324,-8.786638,-6.955195,-1.368712,5.215833,7.502932,5.574240,4.010032,4.945704,5.849448,4.685206
2,0.838221,1.165732,-1.168826,-0.982344,0.180586,1.008354,-0.099242,0.592442,1.265511,1.722809,0.586577,-0.461396,-0.131647,0.381674,1.737487,3.673367,6.150424,8.533894,8.941131,8.750540,7.134045,4.850865,1.447746,-0.295293,-1.393532,-2.280245,-3.578618,-4.053796,-5.805100,-8.891461,-11.979640,-13.155750,-12.770990,-13.203220,-14.202160,-13.528720,-12.121930,-12.766970,-14.940720,-15.772300,...,-5.761971,-3.687450,-1.259601,0.373952,2.022700,4.841165,7.091813,5.870194,0.634342,-4.730296,-7.424703,-7.658523,-7.846692,-7.265346,-5.040391,-2.253080,-1.617007,-3.156716,-3.478971,-0.787138,2.158672,1.380916,-2.976059,-6.804674,-7.088996,-4.791597,-2.294981,-0.087565,2.855343,6.440354,9.171988,9.163836,6.234632,1.857304,-1.697279,-3.090261,-2.469973,-1.284577,-0.743584,-0.966232
3,4.110646,5.326373,5.607666,5.928068,8.450205,10.095110,10.510460,9.351198,8.656458,6.637657,3.696011,1.697838,0.576383,0.511522,1.201125,4.194530,7.390859,9.618025,10.671170,11.257180,10.721330,7.668974,4.521510,1.961572,1.842500,2.354305,3.369698,3.106313,1.564585,-1.749944,-5.112355,-7.318179,-8.420295,-8.656922,-8.217114,-7.594543,-6.677403,-5.767959,-5.266536,-6.005124,...,-12.543650,-9.628821,-6.554149,-5.287622,-7.208587,-10.841520,-12.845910,-10.942060,-6.488643,-3.107108,-3.655696,-7.750552,-11.394100,-11.635640,-8.714692,-6.874737,-7.855653,-8.365008,-4.996743,0.134024,1.006916,-3.861511,-8.585299,-8.348997,-6.015720,-7.181421,-11.427260,-13.052880,-9.535544,-4.653997,-2.141446,-0.119094,3.431744,6.674687,6.437851,4.118158,3.787950,5.909680,6.973948,5.324960
4,-1.787723,-2.623247,-2.818471,-0.157713,-1.216233,-1.771139,-2.252095,-0.006489,-0.408880,0.059605,1.917061,1.906688,1.346496,0.413981,1.654808,0.870527,1.788905,2.318453,3.770809,3.259696,1.886302,2.622774,1.566924,1.850488,-0.078394,-0.595959,-2.783369,-4.299657,-6.760868,-7.901298,-9.015270,-9.458912,-8.514614,-8.791511,-11.150150,-13.143790,-14.089760,-13.926200,-14.371130,-12.867880,...,-15.253650,-9.428393,2.870573,13.407220,20.012770,25.833910,31.118850,29.141530,1

In [ ]:
dataset = dataset.transpose()

In [ ]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
T1,1.308879,4.197934,0.838221,4.110646,-1.787723,-3.965428,3.660656,0.713786,2.177957,-11.92938,0.770604,0.470744,-0.779090,-4.970821,-2.436598,-4.205416,2.348329,4.305004,5.171690
T2,-4.956284,4.799670,1.165732,5.326373,-2.623247,0.425716,1.029767,1.451246,-1.124016,-12.37893,-1.491935,0.855739,-3.951041,-4.888319,-2.572245,-4.350377,7.198255,10.216890,5.866994
T3,-7.472466,5.811594,-1.168826,5.607666,-2.818471,0.674557,0.304995,0.518807,-2.332750,-15.11495,0.739136,3.106529,-4.619559,-5.145712,-1.313881,-4.012769,7.487280,11.520300,8.228498
T4,-4.395810,5.461148,-0.982344,5.928068,-0.157713,-0.965667,1.110863,0.528465,-1.806720,-13.84460,1.546368,2.829681,-4.683345,-4.698628,-1.544094,-3.706159,5.298438,7.640294,6.441741
T5,-3.742904,7.432378,0.180586,8.450205,-1.216233,1.148439,0.175507,0.977232,-4.197453,-11.36576,-0.576602,1.546265,-7.090439,-3.901809,-2.000047,-3.223133,6.940697,6.876572,3.586493


In [ ]:
colum = list()
for x in range(1, 20):
  colum.append(f'P{x}')

In [ ]:
dataset.columns = colum

In [ ]:
dataset.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19
T1,1.308879,4.197934,0.838221,4.110646,-1.787723,-3.965428,3.660656,0.713786,2.177957,-11.92938,0.770604,0.470744,-0.779090,-4.970821,-2.436598,-4.205416,2.348329,4.305004,5.171690
T2,-4.956284,4.799670,1.165732,5.326373,-2.623247,0.425716,1.029767,1.451246,-1.124016,-12.37893,-1.491935,0.855739,-3.951041,-4.888319,-2.572245,-4.350377,7.198255,10.216890,5.866994
T3,-7.472466,5.811594,-1.168826,5.607666,-2.818471,0.674557,0.304995,0.518807,-2.332750,-15.11495,0.739136,3.106529,-4.619559,-5.145712,-1.313881,-4.012769,7.487280,11.520300,8.228498
T4,-4.395810,5.461148,-0.982344,5.928068,-0.157713,-0.965667,1.110863,0.528465,-1.806720,-13.84460,1.546368,2.829681,-4.683345,-4.698628,-1.544094,-3.706159,5.298438,7.640294,6.441741
T5,-3.742904,7.432378,0.180586,8.450205,-1.216233,1.148439,0.175507,0.977232,-4.197453,-11.36576,-0.576602,1.546265,-7.090439,-3.901809,-2.000047,-3.223133,6.940697,6.876572,3.586493


In [ ]:
dataset.shape

(4001, 19)

In [ ]:
# Get Max Value

abs_scaler = MaxAbsScaler()
abs_scaler.fit(dataset)

abs_scaler.max_abs_

array([11.44429, 22.29649, 15.7723 , 20.32594, 31.11885, 21.99364,
       13.63291, 11.30704, 28.64046, 34.41164, 14.28826, 14.04602,
       16.03059, 16.35562, 10.72607, 13.2185 , 16.21857, 16.15717,
       17.29969])

In [ ]:
# Normalizar o dataset

scaled_data = abs_scaler.transform(dataset)

dataset = pd.DataFrame(scaled_data, columns=dataset.columns)


In [ ]:
features = ['P1','P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18']
classe = 'P19'
classes = [classe]

In [ ]:
dataset_only_features = dataset[features]
dataset_only_class = dataset[classes]

In [ ]:
dataset_list = dataset_only_features.stack()
dataset_list

0     P1     0.114370
      P2     0.188278
      P3     0.053145
      P4     0.202236
      P5    -0.057448
               ...   
4000  P14   -0.022766
      P15   -0.041105
      P16   -0.004142
      P17    0.406636
      P18    0.454250
Length: 72018, dtype: float64

In [ ]:
dataset_only_features.head(10)

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18
0,0.114370,0.188278,0.053145,0.202236,-0.057448,-0.180299,0.268516,0.063128,0.076045,-0.346667,0.053933,0.033514,-0.048600,-0.303921,-0.227166,-0.318146,0.144793,0.266445
1,-0.433079,0.215266,0.073910,0.262048,-0.084298,0.019356,0.075535,0.128349,-0.039246,-0.359731,-0.104417,0.060924,-0.246469,-0.298877,-0.239812,-0.329113,0.443828,0.632344
2,-0.652943,0.260651,-0.074106,0.275887,-0.090571,0.030671,0.022372,0.045884,-0.081449,-0.439239,0.051730,0.221168,-0.288171,-0.314614,-0.122494,-0.303572,0.461649,0.713015
3,-0.384105,0.244933,-0.062283,0.291650,-0.005068,-0.043907,0.081484,0.046738,-0.063083,-0.402323,0.108226,0.201458,-0.292151,-0.287279,-0.143957,-0.280377,0.326690,0.472873
4,-0.327054,0.333343,0.011450,0.415735,-0.039083,0.052217,0.012874,0.086427,-0.146557,-0.330288,-0.040355,0.110086,-0.442307,-0.238561,-0.186466,-0.243835,0.427948,0.425605
5,-0.416764,0.397033,0.063932,0.496661,-0.056915,0.164143,-0.113743,0.134552,-0.229027,-0.249186,-0.257820,0.008759,-0.533462,-0.183729,-0.193871,-0.175502,0.566063,0.449940
6,-0.477422,0.465813,-0.006292,0.517096,-0.072371,0.183914,-0.199365,0.019912,-0.245750,-0.254225,-0.294964,0.014171,-0.487227,-0.182777,-0.077257,-0.112095,0.611550,0.435687
7,-0.266839,0.414381,0.037562,0.460062,-0.000209,0.113294,-0.148279,-0.077212,-0.172377,-0.228330,-0.305448,-0.049767,-0.371499,-0.194198,-0.058839,-0.118733,0.527424,0.278828
8,-0.235300,0.390061,0.080236,0.425882,-0.013139,0.144996,-0.138700,-0.203786,-0.139085,-0.221548,-0.304432,-0.066679,-0.363166,-0.210639,-0.024651,-0.156618,0.551442,0.286753
9,-0.302502,0.287536,0.109230,0.326561,0.001915,0.176611,-0.131589,-0.237601,-0.109140,-0.187673,-0.283912,-0.057444,-0.341414,-0.192199,-0.008409,-0.173085,0.530505,0.329965


In [ ]:
dataset_only_class.head()

,P19
0,0.298947
1,0.339139
2,0.475644
3,0.372362
4,0.207315


In [ ]:
# Set valor de sigma para formula da Gaussiana
# 0.3 até 1.5
# 1, 2, 5 - Valores para sigma 
sigma = 0.5

In [ ]:
def calculo_marginais (vector, i, j):

	first_part = (1/(sigma_marginais * math.sqrt(2*math.pi)))

	second_part = math.exp(-(((vector[i] - vector[j])**2)/(2*sigma_marginais**2)))

	return (first_part * second_part)

In [ ]:
# Calcular as marginais
p = dict()
sigma_marginais = 1
list_entropia_marginais = dict() #criar um dicionario para armazenar todos resultados

length = dataset.shape[0]
sqrt
for x in features:
    
    entropia = 0
    
    # Analisa todas as features do dataset
    vector = dataset[x].to_numpy() # converter em numpy array

    entropia = - sum( map(
        lambda i: math.log( sum(  map(lambda j: calculo_marginais(vector, i, j) , range(0,(length)))) / length )
        ,range(0,(length))
        )) / length 
      
      
    list_entropia_marginais[x] =  entropia

In [ ]:
# exibir as 10 primeiras entropias 
for x in range(0, 10):
    print(f'{features[x]} : {list_entropia_marginais[features[x]]}')

P1 : 0.9655570019416212
P2 : 0.9428395311338613
P3 : 0.9774772307270237
P4 : 0.9542655303163872
P5 : 0.9531884811564937
P6 : 0.938131430258612
P7 : 0.9581731699508637
P8 : 0.9746888696030989
P9 : 0.9358881803074196
P10 : 0.9363354585879976


In [ ]:
signal_class = dataset_only_class[classe].to_numpy() # pegar a class

In [ ]:
def calculo_conjunta (vector, class_pos, i, j, M):
  X = vector[i] - vector[j]
  C = class_pos[i] - class_pos[j]

  D = np.array([[X, C]])

  first_part = (1/((2*math.pi) * np.sqrt(np.linalg.det(M))))

  second_part = math.exp(((-1/2) * np.matmul(np.matmul( D, M), D.transpose())))

  return (first_part * second_part)

In [ ]:
# Calcular conjunta

# M = (sigma*np.identity(2))

list_entropia_conjunta = dict() #criar um dicionario para armazenar todos resultados

length = dataset_only_features.shape[0]

for x in features:
    
  entropia = 0
  
  # Analisa todas as features do dataset
  vector = dataset_only_features[x].to_numpy() # converter em numpy array
  
  N = np.array([vector, signal_class])

  
  M = (np.matmul(N, N.transpose())) / length

  M = M / M.max() # Normalizar pelo maior valor da matriz

  entropia =  - sum( map(
		lambda i: math.log(sum ( map(lambda j: calculo_conjunta (vector, signal_class, i, j, M), range(0,(length)))) / length)
		,range(0,(length))
		)) / length

  # Adiciona o calculo de cada features a uma lista
  list_entropia_conjunta[x] = entropia

KeyboardInterrupt: ignored

In [ ]:
# exibir as 10 primeiras entropias conjuntas
for x in range(0, 10):
  print(f'{features[x]} : {list_entropia_conjunta[features[x]]}')

In [ ]:
informacao_mutua = dict()

for x in features:
                           # H(d) + H(c) - H(d,c)
  informacao_mutua[x] = (list_entropia_marginais[x] + list_entropia_marginais[classe] - list_entropia_conjunta[x])

In [ ]:
for x in range(0, 10):
  print(f'{features[x]} : {informacao_mutua[features[x]]}')

In [ ]:
features_selected = list()

features_selected.append(max(informacao_mutua.items(), key=operator.itemgetter(1))[0])

In [ ]:
informacao_mutua[max(informacao_mutua.items(), key=operator.itemgetter(1))[0]]

In [ ]:
max(informacao_mutua.items(), key=operator.itemgetter(1))[0]

In [ ]:
features.remove(max(informacao_mutua.items(), key=operator.itemgetter(1))[0])

In [ ]:
relevance = informacao_mutua
del relevance[max(informacao_mutua.items(), key=operator.itemgetter(1))[0]]

In [ ]:
def calculo_select_features (vector_1, vector_2, i, j, M):
  X = vector_1[i] - vector_1[j]
  Y = vector_2[i] - vector_2[j]

  N = np.array([[X,Y]])

  first_part = 1/((2*math.pi) * np.sqrt(np.linalg.det(M)))

  second_part = math.exp(((-1/2) * np.matmul(np.matmul(N, M), N.transpose())))

  return (first_part * second_part)

In [ ]:
vector_sum = dict()
redun = dict()
#M = (sigma*np.identity(2))

for n in range(0, 9):
  obj1 = relevance
  obj2 = dict()

  for f in features:  
    # Calcular conjunta

    entropia = 0

    # Analisa a ultima feature selecionada do dataset
    last_features_selected = features_selected[-1]
    vector = dataset_only_features[last_features_selected].to_numpy() # converter em numpy array

    N = np.array([vector, dataset_only_features[f]])

    M = (np.matmul(N, N.transpose())) / length

    M = M / M.max() # Normalizar pelo maior valor da matriz 
    
    # calcular a entropia de cada feature
    entropia = - sum(map(lambda i:(
            math.log(sum(map(lambda j:( calculo_select_features (vector, dataset_only_features[f], i, j, M) ), range(0,(length)))) / length )), range(0,(length))))

    
    if f in vector_sum:
      vector_sum[f] = vector_sum[f] + (list_entropia_marginais[last_features_selected] + list_entropia_marginais[f] - entropia)
    else:
      vector_sum[f] = (list_entropia_marginais[last_features_selected] + list_entropia_marginais[f] - entropia)

    redun[f] = vector_sum[f] / len(features_selected)

    obj2[f] = obj1[f]/(redun[f] + 0.0001)

  
  #for x in features:
    #obj2[x] = obj1[x]/(redun[x] + 0.0001)
  
  features_selected.append(max(obj2.items(), key=operator.itemgetter(1))[0]) # adicion as features selecionadas
  features.remove(max(obj2.items(), key=operator.itemgetter(1))[0]) # Removo das features total
  
  del redun[max(obj2.items(), key=operator.itemgetter(1))[0]]
  del vector_sum[max(obj2.items(), key=operator.itemgetter(1))[0]]

  del relevance[max(obj2.items(), key=operator.itemgetter(1))[0]] # Removo da relevance(idleft)


In [ ]:
features_selected